# Guild 관련 feature engineering

In [40]:
from itertools import chain
from tqdm import trange
from tqdm import tqdm
import pickle

# Train

In [2]:
label = pd.read_csv("../data/train_label.csv")
guild = pd.read_csv("../data/train_guild.csv")

In [3]:
print(guild.shape)
guild.tail()

(9963, 2)


,guild_id,guild_member_acc_id
9958,ffe917cf662e746a7491fb55f16151a0f4eff5500b579b...,94eaba795aca5ce53ccbcb8ccf2af788f679f321fd61b3...
9959,ffef2c8316f9aa8e6a29ecd1a0b099bc6f86d4d2048580...,05cd824a467cfc9f8f194c32f735ff52dde8dd898ca532...
9960,ffef446457c4986c7597f680310b347960dfb995a97a1e...,94b8fcf1968f509278e51d7a75e9d22eae0a1d865d9779...
9961,fff2c28b1a1f521eda51809a0568153858dc60b976c291...,293df374edf169385fda206bb90825753186cb0ed65886...
9962,fff5ac3748f18e39c87b31cb52d00b37b4aa205a2b0845...,557f99b8568c83dcf0b6d32f401fbb40b747e1c977b1cb...


## 1. guild count (id 당 guild 참여 횟수) 변수 생성

- guild_member_acc_id에서 예측 대상 유저 id를 추출하여 유저가 몇 개의 guild에 가입되었는지 변수 생성

### 1) guild member acc id 전체를 리스트로 만들기

In [24]:
%%time
guild_acc_id = guild["guild_member_acc_id"].tolist()
guild_acc_id = [x.split(',') for x in guild_acc_id]
guild_id_list = [item for sublist in guild_acc_id for item in sublist]

CPU times: user 21.5 ms, sys: 4.33 ms, total: 25.8 ms
Wall time: 24.8 ms


In [25]:
len(guild_id_list)

62529

In [26]:
len(list(set(guild_id_list)))

40836

### 2) train acc_id당 guild_count(길드 참여횟수) 변수 생성

In [27]:
%%time
df_guild_id = pd.DataFrame(guild_id_list, columns=["acc_id"])
df_guild_id = df_guild_id.groupby('acc_id').size().reset_index(name='guild_counts')
guild_1 = pd.merge(guild_1, df_guild_id, how='left')
guild_1["guild_counts"] = guild_1["guild_counts"].fillna(0)

CPU times: user 130 ms, sys: 9.43 ms, total: 140 ms
Wall time: 139 ms


In [28]:
guild_1.head(10)

,acc_id,label,guild_total_member_count,guild_counts
0,b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...,week,0.0,0.0
1,ed500c4957956b3e99dc3985666850b582f812405eefb6...,week,0.0,0.0
2,acc6afa23a6bf15e18151e4794c7789225ef9d682f473c...,week,0.0,0.0
3,34095a3c9a2937ced3ea3fd75e22ce177dc5879d2a53f7...,week,0.0,0.0
4,26f3db6e8817a93c4ceda9a16f0832945e43d950b95882...,week,0.0,0.0
5,aaf2fb67be2d51bf75b973df5587dda57aa147c123c2a4...,week,0.0,0.0
6,bc5878e17e1673e7882c5c325a6fb22940a5e5330092cd...,week,49.0,1.0
7,2beb296998ecb631341c0469a5a4a7c5e9e7ce1f52389a...,week,0.0,0.0
8,69f05d55892e869274c2f49061d93addf8c58fe1865e86...,week,0.0,0.0
9,ccb31ea10a8b9a54ff5c853fd1e19db3e25ae88adfc08a...,week,0.0,0.0


In [29]:
guild_1.describe(include="all")

,acc_id,label,guild_total_member_count,guild_counts
count,100000,100000,100000.000000,100000.000000
unique,100000,4,NaN,NaN
top,11348acf24e6bef9e91a5500cb4602580616e95213174a...,retained,NaN,NaN
freq,1,25000,NaN,NaN
mean,NaN,NaN,4.440180,0.272380
std,NaN,NaN,13.425615,0.740564
min,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,0.000000,0.000000
50%,NaN,NaN,0.000000,0.000000
75%,NaN,NaN,0.000000,0.000000


## 2. guild size (guild 참여 id 수) 변수 생성

- 유저가 가입된 guild에 몇 명의 멤버가 가입되어 있는지 guild size를 구함 (복수의 guild에 가입되어 있을 경우 최댓값)

### 1) 각 guild의 size 구하기
- guild size: 1명~77명

In [30]:
guild["guild_size"] = guild["guild_member_acc_id"].apply(lambda x: x.count(",") + 1)

In [31]:
guild.describe(include="all")

,guild_id,guild_member_acc_id,guild_member_count,guild_size
count,9963,9963,9963.000000,9963.000000
unique,9963,9341,NaN,NaN
top,2b7ae5949cfc3d089ea2932da18823c9abdd192d840152...,d36a8c1b5d862572a31a9e1c816520c1fc93577ddd8cd4...,NaN,NaN
freq,1,10,NaN,NaN
mean,NaN,NaN,6.276122,6.276122
std,NaN,NaN,8.705304,8.705304
min,NaN,NaN,1.000000,1.000000
25%,NaN,NaN,1.000000,1.000000
50%,NaN,NaN,2.000000,2.000000
75%,NaN,NaN,8.000000,8.000000


### 2) id가 속한 guild의 size 변수 생성
- 여러개 guild에 가입한 경우 가장 큰 guild의 사이즈 선택

In [35]:
def get_guild_size(x):
    id_guild_size = []
    for i in range(len(guild)):
        if guild.loc[i,"guild_member_acc_id"].count(x) == 1:
            id_guild_size.append(guild.loc[i,"guild_size"])
    return(max(id_guild_size))

In [36]:
guild_1["guild_size"] = 0

In [37]:
for i in tqdm(range(len(guild_1))):
    if guild_1.loc[i,"guild_counts"] == 0:
        pass
    else:
        guild_1.loc[i,"guild_size"] = get_guild_size(guild_1.loc[i,"acc_id"])

100%|██████████| 100000/100000 [22:12<00:00, 75.02it/s]


In [38]:
guild_1.describe()

,guild_total_member_count,guild_counts,guild_size
count,100000.000000,100000.000000,100000.000000
mean,4.440180,0.272380,3.625280
std,13.425615,0.740564,10.404309
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,187.000000,11.000000,77.000000


In [41]:
pickle.dump(guild_1,open('../data/merged_train_guild.pkl','wb'))

In [42]:
merged_train_guild = pickle.load(open('../data/merged_train_guild.pkl','rb'))

In [43]:
merged_train_guild.tail()

,acc_id,label,guild_total_member_count,guild_counts,guild_size
99995,da6d33b03968d8e35821f6eb88ad22e12e37aa8867084e...,retained,76.0,2.0,51
99996,676c944f4b6ae63818b3cad824a61233690f16a2275d5d...,retained,0.0,0.0,0
99997,695e1f28e234fc4cc53085e332fa7a76d7895ca4cc745b...,retained,0.0,0.0,0
99998,0c87fabaad5542e533f958a1d6fd739993b94e95e00989...,retained,0.0,0.0,0
99999,47ff575cb94019df5695c5d81ec285b0d801607b2a8697...,retained,24.0,1.0,24


## 3. guild total member count

- 유저가 가입된 guild의 멤버수를 모두 합(여러개의 guild에 가입한 유저 존재)한 변수 생성

In [4]:
%%time
guild_member_acc_id_length_list = []
for guild_member_list in guild['guild_member_acc_id']:
    guild_member_acc_id_length_list.append(len(guild_member_list.split(',')))

CPU times: user 8.71 ms, sys: 277 µs, total: 8.99 ms
Wall time: 8.94 ms


In [5]:
guild_member_count = pd.DataFrame(guild_member_acc_id_length_list, columns=['guild_member_count'])
guild = pd.concat([guild, guild_member_count],axis=1)

In [6]:
%%time

guild_member_lists = []
for i in range(len(guild['guild_member_acc_id'])):
    guild_member_lists = guild_member_lists + guild['guild_member_acc_id'][i].split(',')

CPU times: user 2.55 s, sys: 48.3 ms, total: 2.6 s
Wall time: 2.6 s


In [7]:
guild_member_count = [[guild['guild_member_count'][i]]*guild['guild_member_count'][i] for i in trange(len(guild))]

100%|██████████| 9963/9963 [00:00<00:00, 38388.16it/s]


In [8]:
flat_guild_member_count = list(chain.from_iterable(guild_member_count))

In [9]:
%%time
all_id_and_guild_member_count_df = \
pd.concat([pd.Series(guild_member_lists), pd.Series(flat_guild_member_count)], axis=1)
all_id_and_guild_member_count_df.columns = ['acc_id','guild_member_count']

CPU times: user 16 ms, sys: 1.8 ms, total: 17.8 ms
Wall time: 16.9 ms


In [10]:
%%time 
guild_member_count = \
all_id_and_guild_member_count_df.groupby('acc_id')['guild_member_count'].sum()
guild_member_count_df = pd.DataFrame(guild_member_count).reset_index()
guild_member_count_df.columns = ['acc_id','guild_total_member_count']

CPU times: user 71 ms, sys: 2.28 ms, total: 73.3 ms
Wall time: 74.3 ms


In [22]:
guild_1 = pd.merge(label, guild_member_count_df, how='left')
guild_1.fillna(0,inplace=True)

In [23]:
guild_1.tail()

,acc_id,label,guild_total_member_count
99995,da6d33b03968d8e35821f6eb88ad22e12e37aa8867084e...,retained,76.0
99996,676c944f4b6ae63818b3cad824a61233690f16a2275d5d...,retained,0.0
99997,695e1f28e234fc4cc53085e332fa7a76d7895ca4cc745b...,retained,0.0
99998,0c87fabaad5542e533f958a1d6fd739993b94e95e00989...,retained,0.0
99999,47ff575cb94019df5695c5d81ec285b0d801607b2a8697...,retained,24.0


---

# Test

In [44]:
activity = pd.read_csv('../data/test_activity.csv')
label = pd.DataFrame(list(activity['acc_id'].unique()))
label.columns = ['acc_id']

guild = pd.read_csv("../data/test_guild.csv")

In [45]:
print(guild.shape)
guild.tail()

(5906, 2)


,guild_id,guild_member_acc_id
5901,ffc5e3081eb0e2dacfaa5764ad6b1217b3ce5ddb58901f...,d9b65373063af0235d4671c81845341d8f6fd30b2a2afa...
5902,ffcd0dc7f52afa225c4704b152a8f23ed7b31e0c65cc23...,702430f360b001863d1c9819b2b343cee913051bd0f485...
5903,ffe737564aaf515dd77b516dfed4695794c42e8b2a3837...,b9387283921208fb8c40692c9ed0614f4679d8d8454875...
5904,ffef2c8316f9aa8e6a29ecd1a0b099bc6f86d4d2048580...,05cd824a467cfc9f8f194c32f735ff52dde8dd898ca532...
5905,fff5ac3748f18e39c87b31cb52d00b37b4aa205a2b0845...,557f99b8568c83dcf0b6d32f401fbb40b747e1c977b1cb...


## 1. guild count (id 당 guild 참여 횟수) 변수 생성

- guild_member_acc_id에서 예측 대상 유저 id를 추출하여 유저가 몇 개의 guild에 가입되었는지 변수 생성

### 1) guild member acc id 전체를 리스트로 만들기

In [55]:
%%time
guild_acc_id = guild["guild_member_acc_id"].tolist()
guild_acc_id = [x.split(',') for x in guild_acc_id]
guild_id_list = [item for sublist in guild_acc_id for item in sublist]

CPU times: user 17.5 ms, sys: 2.51 ms, total: 20 ms
Wall time: 19.2 ms


In [56]:
len(guild_id_list)

51648

In [57]:
len(list(set(guild_id_list)))

36468

### 2) train acc_id당 guild_count(길드 참여횟수) 변수 생성

In [58]:
%%time
df_guild_id = pd.DataFrame(guild_id_list, columns=["acc_id"])
df_guild_id = df_guild_id.groupby('acc_id').size().reset_index(name='guild_counts')
guild_1 = pd.merge(guild_1, df_guild_id, how='left')
guild_1["guild_counts"] = guild_1["guild_counts"].fillna(0)

CPU times: user 110 ms, sys: 9.31 ms, total: 119 ms
Wall time: 119 ms


In [59]:
guild_1.head(10)

,acc_id,guild_total_member_count,guild_counts
0,6ac252acf5ce113d2eb251d62ee2713c269655be3f8bec...,0.0,0.0
1,be14d8ed6f435440d8b8e50b3997a30682a113458204e4...,0.0,0.0
2,911660b6ab8860b6d27fe2ff17d715938b33f261919bf1...,37.0,2.0
3,9f8fefd8917c0406b108e95cb578d46ea9b360e39200e9...,0.0,0.0
4,6a3f52c863e97184ee8487c34b59c364e8427724aa29b8...,20.0,1.0
5,b05933d268fdf4c92fd6331992f2ecda41e1c28d12eb40...,0.0,0.0
6,14cf6302887cd135a6a312caace5608af778e40a17e889...,0.0,0.0
7,1f440d69c125b3d99f85add756cbbe1e4e46921fb16dfd...,1.0,1.0
8,fe2a2e260e89ae26684ec33ebb239a52b366f4db05553f...,19.0,3.0
9,0662a8c684c6de4f0f92012409a48f5340658e21ea5078...,0.0,0.0


In [60]:
guild_1.describe(include="all")

,acc_id,guild_total_member_count,guild_counts
count,40000,40000.000000,40000.000000
unique,40000,NaN,NaN
top,61080f48b9dce4432e6a17097e5ccfab46036c9b64f05e...,NaN,NaN
freq,1,NaN,NaN
mean,NaN,4.439425,0.274650
std,NaN,13.502198,0.748185
min,NaN,0.000000,0.000000
25%,NaN,0.000000,0.000000
50%,NaN,0.000000,0.000000
75%,NaN,0.000000,0.000000


## 2. guild size (guild 참여 id 수) 변수 생성

- 유저가 가입된 guild에 몇 명의 멤버가 가입되어 있는지 guild size를 구함 (복수의 guild에 가입되어 있을 경우 최댓값)

### 1) 각 guild의 size 구하기
- guild size: 1명~77명

In [61]:
guild["guild_size"] = guild["guild_member_acc_id"].apply(lambda x: x.count(",") + 1)

In [62]:
guild.describe(include="all")

,guild_id,guild_member_acc_id,guild_member_count,guild_size
count,5906,5906,5906.000000,5906.000000
unique,5906,5629,NaN,NaN
top,097f04bc292c182c61bb8fc448957235e95008a70dcda0...,a362f182cd074a083b40164257a965ef184ecd9947b7a9...,NaN,NaN
freq,1,8,NaN,NaN
mean,NaN,NaN,8.745005,8.745005
std,NaN,NaN,10.337423,10.337423
min,NaN,NaN,1.000000,1.000000
25%,NaN,NaN,1.000000,1.000000
50%,NaN,NaN,4.000000,4.000000
75%,NaN,NaN,12.000000,12.000000


### 2) id가 속한 guild의 size 변수 생성
- 여러개 guild에 가입한 경우 가장 큰 guild의 사이즈 선택

In [63]:
def get_guild_size(x):
    id_guild_size = []
    for i in range(len(guild)):
        if guild.loc[i,"guild_member_acc_id"].count(x) == 1:
            id_guild_size.append(guild.loc[i,"guild_size"])
    return(max(id_guild_size))

In [64]:
guild_1["guild_size"] = 0

In [65]:
for i in tqdm(range(len(guild_1))):
    if guild_1.loc[i,"guild_counts"] == 0:
        pass
    else:
        guild_1.loc[i,"guild_size"] = get_guild_size(guild_1.loc[i,"acc_id"])

100%|██████████| 40000/40000 [05:06<00:00, 130.30it/s]


In [66]:
guild_1.describe()

,guild_total_member_count,guild_counts,guild_size
count,40000.000000,40000.000000,40000.000000
mean,4.439425,0.274650,3.618600
std,13.502198,0.748185,10.394875
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,296.000000,11.000000,77.000000


In [67]:
pickle.dump(guild_1,open('../data/merged_test_guild.pkl','wb'))

In [68]:
merged_test_guild = pickle.load(open('../data/merged_test_guild.pkl','rb'))

In [69]:
merged_test_guild.tail()

,acc_id,guild_total_member_count,guild_counts,guild_size
39995,d048b24f5205a50916339d24fc9f7b1c69352d2ee10f1c...,0.0,0.0,0
39996,ad5b3ef19e64aceb9ffea55310607ba62146218e616a83...,0.0,0.0,0
39997,972114a3bb57c377d162fb66b9198a2eec40de9576af51...,0.0,0.0,0
39998,3d10d680df3d5fa2e370997c18274bad7355e3d0ff9fb0...,67.0,3.0,45
39999,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,0.0,0.0,0


## 3. guild total member count

- 유저가 가입된 guild의 멤버수를 모두 합(여러개의 guild에 가입한 유저 존재)한 변수 생성

In [46]:
%%time
guild_member_acc_id_length_list = []
for guild_member_list in guild['guild_member_acc_id']:
    guild_member_acc_id_length_list.append(len(guild_member_list.split(',')))

CPU times: user 8.08 ms, sys: 581 µs, total: 8.66 ms
Wall time: 8.22 ms


In [47]:
guild_member_count = pd.DataFrame(guild_member_acc_id_length_list, columns=['guild_member_count'])
guild = pd.concat([guild, guild_member_count],axis=1)

In [48]:
%%time

guild_member_lists = []
for i in range(len(guild['guild_member_acc_id'])):
    guild_member_lists = guild_member_lists + guild['guild_member_acc_id'][i].split(',')

CPU times: user 883 ms, sys: 25 ms, total: 908 ms
Wall time: 906 ms


In [49]:
guild_member_count = [[guild['guild_member_count'][i]]*guild['guild_member_count'][i] for i in trange(len(guild))]

100%|██████████| 5906/5906 [00:00<00:00, 36957.47it/s]


In [50]:
flat_guild_member_count = list(chain.from_iterable(guild_member_count))

In [51]:
%%time
all_id_and_guild_member_count_df = \
pd.concat([pd.Series(guild_member_lists), pd.Series(flat_guild_member_count)], axis=1)
all_id_and_guild_member_count_df.columns = ['acc_id','guild_member_count']

CPU times: user 18.8 ms, sys: 2.63 ms, total: 21.4 ms
Wall time: 20.5 ms


In [52]:
%%time 
guild_member_count = \
all_id_and_guild_member_count_df.groupby('acc_id')['guild_member_count'].sum()
guild_member_count_df = pd.DataFrame(guild_member_count).reset_index()
guild_member_count_df.columns = ['acc_id','guild_total_member_count']

CPU times: user 61.6 ms, sys: 2.89 ms, total: 64.5 ms
Wall time: 63 ms


In [53]:
guild_1 = pd.merge(label, guild_member_count_df, how='left')
guild_1.fillna(0,inplace=True)

In [54]:
guild_1.tail()

,acc_id,guild_total_member_count
39995,d048b24f5205a50916339d24fc9f7b1c69352d2ee10f1c...,0.0
39996,ad5b3ef19e64aceb9ffea55310607ba62146218e616a83...,0.0
39997,972114a3bb57c377d162fb66b9198a2eec40de9576af51...,0.0
39998,3d10d680df3d5fa2e370997c18274bad7355e3d0ff9fb0...,67.0
39999,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,0.0
